In [ ]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENAI_TEMPERATURE = float(os.getenv("OPENAI_TEMPERATURE", 0.0))


openai.api_key = OPENAI_API_KEY
print(OPENAI_API_KEY)

In [ ]:

import json
from pathlib import Path
from pprint import pprint

import pandas as pd
df = pd.read_json('/Users/kim-yeongsang/Desktop/gme/instructino_ai/data_cbnu_rag_question.json')
df.head()

In [ ]:
df = df.dropna(how='any', axis= 0)

In [ ]:
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

def get_embedding(text, model="text-embedding-ada-002"):
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

df['question_vector'] = df.question.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
df.head()

In [ ]:
df.head()

In [ ]:
df["vector_id"] = range(0, len(df))
df = df.astype({'vector_id':'str'})
df.head()

In [ ]:
df.to_json('data_rag_question_embedding_.json', force_ascii=False, orient = 'records',indent=4)

In [ ]:
import chromadb

chroma_client = chromadb.Client()

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.

# Note. alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")


embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name='text-embedding-ada-002')
question_collection = chroma_client.create_collection(name='quesiton', embedding_function=embedding_function)

In [ ]:
batch_size = 166
chunks = [df[i:i + batch_size] for i in range(0, len(df), batch_size)]

for chunk in chunks:
    question_collection.add(
        ids=chunk['vector_id'].tolist(),
        embeddings=chunk['question_vector'].tolist(),  # Assuming you have the 'question_vector' column
    )

In [ ]:
def query_collection(collection, query, max_results, dataframe):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':results['distances'][0],
                'fileName': dataframe[dataframe.vector_id.isin(results['ids'][0])]['fileName'],
                'content': dataframe[dataframe.vector_id.isin(results['ids'][0])]['content'],
                'question': dataframe[dataframe.vector_id.isin(results['ids'][0])]['question'],
                })
    return df

In [ ]:
LLM_MODEL = os.getenv("LLM_MODEL", os.getenv("OPENAI_API_MODEL", "gpt-3.5-turbo")).lower()
OPENAI_TEMPERATURE = float(os.getenv("OPENAI_TEMPERATURE", 0.0))


In [ ]:
import tiktoken as tiktoken
import time


def limit_tokens_from_string(string: str, model: str, limit: int) -> str:
    """Limits the string to a number of tokens (estimated)."""

    try:
        encoding = tiktoken.encoding_for_model(model)
    except:
        encoding = tiktoken.encoding_for_model('gpt2')  # Fallback for others.

    encoded = encoding.encode(string)

    return encoding.decode(encoded[:limit])

In [ ]:
def openai_call(
    system_prompt: str,
    prompt: str,
    model: str = LLM_MODEL,
    temperature: float = OPENAI_TEMPERATURE,
    max_tokens: int = 100,
):
    while True:
        try:
            if not model.lower().startswith("gpt-"):
                # Use completion API
                response = openai.Completion.create(
                    engine=model,
                    prompt=prompt,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=1,
                    frequency_penalty=0,
                    presence_penalty=0,
                )
                return response.choices[0].text.strip()
            else:
                # Use 4000 instead of the real limit (4097) to give a bit of wiggle room for the encoding of roles.
                # TODO: different limits for different models.

                trimmed_prompt = limit_tokens_from_string(prompt, model, 4000 - max_tokens)

                # Use chat completion API
                messages = [{"role": "system" , "content": system_prompt},{"role": "system", "content": trimmed_prompt}]
                response = openai.ChatCompletion.create(
                    model='gpt-3.5-turbo',
                    messages=messages,
                    temperature=temperature,
                    max_tokens=2000,
                    n=1,
                    stop=None,
                )    
                return response.choices[0].message.content.strip()
        except openai.error.RateLimitError:
            print(
                "   *** The OpenAI API rate limit has been exceeded. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.Timeout:
            print(
                "   *** OpenAI API timeout occurred. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIError:
            print(
                "   *** OpenAI API error occurred. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIConnectionError:
            print(
                "   *** OpenAI API connection error occurred. Check your network settings, proxy configuration, SSL certificates, or firewall rules. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.InvalidRequestError:
            print(
                "   *** OpenAI API invalid request. Check the documentation for the specific API method you are calling and make sure you are sending valid and complete parameters. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.ServiceUnavailableError:
            print(
                "   *** OpenAI API service unavailable. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        else:
            break

In [ ]:
question = '..'


title_query_result = query_collection(
    collection=question_collection,
    query=question,
    max_results=5,
    dataframe=df
)